In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import json
import pandas as pd
import time
def nothing(x):
    pass

In [6]:
def get_vortices_coordinates(mask, frame):
    _, contours, hierarchy= cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    vortices_list = []
    for (i,c) in enumerate(contours):
        cx= c[0][0][0]
        cy= c[0][0][1]
        vortices_list.append((cx, cy))
    return vortices_list

# EXTRACTION MOMENTS OF TIME FOR WHICH THE DATA IS LABELED

In [7]:
''' EXTRACT TIME RANGE '''
mypath = 'CalculateDetectionMetrics/cropped_data_labeled/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
time_range = np.array(sorted([float(x[19:-5]) for x in list(filter(lambda s: 'json' in s, onlyfiles))]))
time_range = time_range[time_range >= 30]
time_range = np.round(time_range, 1)
time_range

array([ 30. ,  33. ,  36. ,  39. ,  42.1,  45. ,  48. ,  51. ,  54. ,
        57. ,  60. ,  63. ,  66. ,  69. ,  72. ,  75. ,  78. ,  81. ,
        84. ,  87. ,  90. ,  93. ,  96. ,  99. , 102. , 105. , 108. ,
       111. , 114. , 117. , 120. , 123. , 126. , 129. , 132. , 135. ,
       138. , 141. , 144. , 147. , 150. , 153. , 156. , 159. , 162. ,
       165. , 168. , 171. , 174. , 177. , 180. , 183. , 186. , 189. ,
       192. , 195. , 198. , 201. , 204. , 207. , 210.1, 214. , 217. ,
       220. , 223. , 226. , 229. , 232. , 235. , 238. , 241. , 244. ,
       247. , 250. , 253. , 256. , 259. , 262. , 265. , 268. , 271. ,
       274. , 277. , 280. , 283. , 286. , 289. , 292. , 295. , 298. ,
       301. , 304. , 307. , 310. , 313. , 316. , 319. , 322. , 325. ,
       328. ])

# METHOD 1 ALGORITHM

In [8]:
program_starts = time.time()

detected_vortices = {}
for t in time_range:
    # cv2.namedWindow("Tracking")
    # cv2.createTrackbar("LH", "Tracking", 0, 255, nothing)
    # cv2.createTrackbar("LS", "Tracking", 0, 255, nothing)
    # cv2.createTrackbar("LV", "Tracking", 0, 255, nothing)
    # cv2.createTrackbar("UH", "Tracking", 255, 255, nothing)
    # cv2.createTrackbar("US", "Tracking", 255, 255, nothing)
    # cv2.createTrackbar("UV", "Tracking", 255, 255, nothing)

    frame = cv2.imread(f'CalculateDetectionMetrics/cropped_data_labeled/x_vel_cropped_time={np.round(t, 1)}0.png')
    # frame = cv2.resize(frame, (817,817))

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    l_h = 88 #cv2.getTrackbarPos("LH", "Tracking")
    l_s = 158 #cv2.getTrackbarPos("LS", "Tracking")
    l_v = 0 #cv2.getTrackbarPos("LV", "Tracking")

    u_h = 255 #cv2.getTrackbarPos("UH", "Tracking")
    u_s = 255 #cv2.getTrackbarPos("US", "Tracking")
    u_v = 255 #cv2.getTrackbarPos("UV", "Tracking")

    l_b = np.array([l_h, l_s, l_v])
    u_b = np.array([u_h, u_s, u_v])

    mask = cv2.inRange(hsv, l_b, u_b)

    #     res = cv2.bitwise_and(frame, frame, mask=mask)

    ''' LABEL DETECTED OBJECTS + GET COORDINATES'''
    detected_vortices[np.round(t,1)] = get_vortices_coordinates(mask, frame)

now = time.time()
print("It has been {0} seconds since the loop started".format(now - program_starts))

It has been 4.110339164733887 seconds since the loop started


# TP, FP, FN METRICS CALCULATIONS

In [11]:
''' READ JSON FILES WITH LABELED BOUNDING BOXES '''
bbox_size = 3
for index, t in enumerate(time_range):
    with open(f'CalculateDetectionMetrics/cropped_data_labeled/x_vel_cropped_time={np.round(t,1)}0.json', 'r') as f:
        dictData = json.load(f)
    frame = cv2.imread(f'CalculateDetectionMetrics/cropped_data_labeled/x_vel_cropped_time={np.round(t,1)}0.png')
    TP = []
    for i, vortex in enumerate(dictData['shapes']):
        labeled_mask = np.zeros((frame.shape[:2]), dtype='uint8')
        a = int(vortex['points'][0][0])
        b = int(vortex['points'][0][1])
        c = int(vortex['points'][1][0])
        d = int(vortex['points'][1][1])
    #         print(f'vortex {i+1}', a, b, c, d)
        cv2.rectangle(labeled_mask, (a,b), (c,d), 255, cv2.FILLED)
        for x,y in detected_vortices[np.round(t,1)]:
            if (x,y) not in TP:
                detected_vortex = np.zeros((frame.shape[:2]), dtype='uint8')
                cv2.rectangle(detected_vortex, (x-bbox_size,y-bbox_size), (x+bbox_size,y+bbox_size), 255, cv2.FILLED)
                result = np.all(cv2.bitwise_and(detected_vortex, labeled_mask) == 0)
                if not result:
                    TP.append((x,y))
                    break
    tp_count = len(TP) #CORRECT
    fp_count = len(detected_vortices[np.round(t,1)]) - len(TP) #INCORRECT
    fn_count = len(dictData['shapes']) - len(TP) #UNDETECTED
    
    ''' INSPECTION IF CALCULATION WORK PROPERLY '''
    if index % 10 == 0:
        for (x,y) in detected_vortices[np.round(t,1)]:
            if (x,y) in TP:
                color = (0,255,0)
            else:
                color = (0,0,255)
            cv2.rectangle(frame, (x-12,y-12), (x+12,y+12), color, 2)
        for parameter, value, shift in zip(['T=', 'TP=', 'FP=', 'FN='], [t, tp_count, fp_count, fn_count], [30, 60, 90, 120]):
            cv2.putText(frame, text=parameter+str(value), org=(10, shift),
                    fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.9, color=(0,0,0),
                    thickness=1, lineType=cv2.LINE_AA)
        cv2.imwrite(f'Method1-ImageThresholding_RESULTS/TP_FP_{np.round(t,1)}.png', frame)
        
    ''' SAVE RESULTS TO FILE '''
#     print(t, tp_count, fp_count, fn_count)
    with open('Method1-ImageThresholding_RESULTS/Method1-Detection-Metrics-Results.csv', 'a') as fr:
        fr.write(f'{t}\t{tp_count}\t{fp_count}\t{fn_count}\n')

# FINAL METRICS SUMMATION

In [33]:
df_metrics = pd.read_csv('Method1-ImageThresholding_RESULTS/Method1-Detection-Metrics-Results.csv', header=None, delimiter='\t')
df_metrics.columns = ['TIME', 'TP', 'FP', 'FN']
df_metrics.set_index('TIME', inplace=True)
df_metrics

,TP,FP,FN
TIME,,,
30.0,5,4,0
33.0,5,6,1
36.0,5,5,1
39.0,3,4,2
42.1,2,1,0
...,...,...,...
316.0,41,33,3
319.0,40,23,2
322.0,46,32,2


In [38]:
df_metrics.sum()

TP    2089
FP    1950
FN     264
dtype: int64

# SAVE TO FILE DETECTED COORDINATES

In [9]:
with open('Method1-ImageThresholding_RESULTS/detected_vortices.txt','w') as data: 
      data.write(str(detected_vortices))